# Penitent Nomad

### Goal
```QUESTION ```I want to know how many areas in the United States are within so many miles of different stores. For example: Which areas in the United States are within 100 miles of an IKEA and 30 miles of a Micro Center? What if I wanted areas also within 50 miles of Store C and 20 miles of Store D? 

 ```STORES ```The criteria I used for picking stores was first - if it was a store that I enjoyed visiting and second - stores with a relatively limited number of locations, since many larger retail chains are ubiquitous in most or all regions of the country.

 ```DATA ```All store data was gathered by myself and was either scraped directly from their websites using python's ```sracpy``` package or found on an open API endpoint. Data was organized and cleaned using a combination of custom python/javascript scripts and Google Sheets.

In [27]:
import sys
from typing import List, Tuple, Dict, Any, Hashable
import os
import math
import time
import pandas as pd
import numpy as np
import folium
import itertools as it
import great_circle_calculator.great_circle_calculator as gcc
import logging

sys.path.insert(0, "..")
from helpers import nomad as nom

def myLogger():
    logger = logging.getLogger(__name__)
    formatter = logging.Formatter(
        "%(asctime)s: %(lineno)2s: %(levelname)s: %(message)s"
    )
    logger.setLevel(logging.DEBUG)
    logger.propagate = False
    filename = "nomad_intersections.log"
    if logger.hasHandlers():
        logger.handlers.clear()

    fhandler = logging.FileHandler(filename=filename, mode="a")
    fhandler.setFormatter(formatter)
    fhandler.setLevel(logging.ERROR)
    logger.addHandler(fhandler)

    consoleHandler = logging.StreamHandler(sys.stdout)
    consoleHandler.setFormatter(formatter)
    consoleHandler.setLevel(logging.DEBUG)
    logger.addHandler(consoleHandler)

    return logger

logger = myLogger()

## Importing CSVs

In [28]:
stores_dataframe = pd.read_csv(
    "https://raw.githubusercontent.com/voxxtelum/penitent_nomad/main/common/csv/merged_locations.csv",
    dtype={"store_zip": "str"},
)

logger.debug(f"Unique short_name found: {stores_dataframe.short_name.unique()}")
stores_dataframe.tail(5)

2022-10-25 14:24:30,443:  6: DEBUG: Unique short_name found: ['containerstore' 'ikea' 'microcenter' 'llbean']


,store_uid,uuid,short_name,store_name,store_desc,store_add1,store_add2,store_city,store_state,store_zip,store_address_full,Latitude,Longitude,rand_uuid
222,04032_32866f06_llbean_freeport_outlet_store,32866f06,llbean,LL Bean,Freeport Outlet Store,1 Freeport Village Station,Village Station,Freeport,ME,04032,"1 Freeport Village Station Freeport, ME 04032",43.855819,-70.102834,ab6af5ea
223,02649_fb43c61a_llbean_mashpee,fb43c61a,llbean,LL Bean,Mashpee,7 Market Street,Mashpee Commons,Mashpee,MA,02649,"7 Market Street Mashpee, MA 02649",41.617410,-70.490694,ad8a444c
224,04401_c1b75319_llbean_bangor_outlet_store,c1b75319,llbean,LL Bean,Bangor Outlet Store,534 Stillwater Avenue,Parkade Shopping Center,Bangor,ME,04401,"534 Stillwater Avenue Bangor, ME 04401",44.830215,-68.756095,23eee280
225,04605_6348fda9_llbean_ellsworth_outlet_store,6348fda9,llbean,LL Bean,Ellsworth Outlet Store,150 High Street,NaN,Ellsworth,ME,04605,"150 High Street Ellsworth, ME 04605",44.537095,-68.411994,29b7344b
226,84060_34caa27c_llbean_park_city,34caa27c,llbean,LL Bean,Park City,675 Main Street,The Kimball On Main,Park City,UT,84060,"675 Main Street Park City, UT 84060",40.646456,-111.497615,bf8a61b3


## Global Settings

In [29]:
# Earth radius in miles
earth_r = 3958.8

# Store settings must be added. Radius in miles
stores_settings = {
    "ikea": {"radius": 50, "enabled": True},
    "microcenter": {"radius": 30, "enabled": True},
    "containerstore": {"radius": 15, "enabled": True},
    "llbean": {"radius": 25, "enabled": True},
}

## Transforming Data
### Transformed Dataframe
First convert Latitude and Longitude headers to a list of (Lon, Lat) coordinates.

Then add radius and enabled from store settings and sort the dataframe by radius.


In [30]:
stores_dataframe["coordinates"] = list(
    zip(stores_dataframe.Longitude, stores_dataframe.Latitude)
)

stores_dataframe_c = stores_dataframe.copy()

stores_dataframe_c = (
    stores_dataframe[["short_name", "coordinates"]]
    .groupby("short_name")
    .agg({"coordinates": lambda c: list(c)})
    .reset_index()
)

stores_dataframe_c = stores_dataframe_c.rename(columns={"short_name": "store"})
stores_dataframe_c["radius"] = stores_dataframe_c.apply(
    lambda row: stores_settings[row["store"]]["radius"], axis=1
)
stores_dataframe_c["enabled"] = stores_dataframe_c.apply(
    lambda row: stores_settings[row["store"]]["enabled"], axis=1
)
stores_dataframe_c = stores_dataframe_c.sort_values(
    by="radius", ascending=False, ignore_index=True
)

stores_dataframe_c.head()

,store,coordinates,radius,enabled
0,ikea,"[(-71.0684562, 42.1371268), (-72.9200335, 41.2...",50,True
1,microcenter,"[(-71.1143075, 42.3572095), (-73.5858212, 40.7...",30,True
2,llbean,"[(-88.163882, 43.035436), (-89.453067, 43.0718...",25,True
3,containerstore,"[(-70.9445901, 42.5414196), (-71.1687096, 42.3...",15,True


### Converting Dataframe to Records

In [31]:
stores_dataframe_d = stores_dataframe_c[stores_dataframe_c["enabled"] == True]
stores_dataframe_d = stores_dataframe_d.to_dict("records")


## Starting the Loop
### Recursive Function

```OUT ```Local Points Dataframe

Local Points are the coordinates of each store that match all of the criteria. For example: one cluster will include the first location with all subsequent locations of the second store within the radius of the first location, then all locations of the third store within the radius of the second location, and so on.

```OUT ```Intermediate Points Dataframe

Intermediate Points are point along a line drawn between each local point's center. Rather than finding a midpoint, the distance is weighted based on the radius set for the two stores, and favors stores with smaller radii. I do this because if I wanted to be within R=100 miles of one location and R=25 miles of another location, rather than centering around a midway point, I would rather be closer to the store with the smaller radius setting.

In [32]:
def int_cluster_points_multi(stores: List[Dict[Hashable, Any]]):
    """Sets first store in list (should be sorted by largest radius) as anchor with coord and radius, then searches in anchor radius
    to find all targets that intersect with circle of target radius and center of target coordinate.
    Then finds all subsequent locations around each target location, and calculates intermediate points on each line intersecting
    each locations centers.
    """

    anchor = stores[0]
    anchor_points = anchor["coordinates"]
    target = stores[1]
    targets = stores[1:]
    anchor_radius = anchor["radius"]
    targets_count = len(targets)

    logger.debug(
        f'Using Store: ({len(anchor["coordinates"])}) {anchor["store"].upper()} Locations as Anchor Points with Radius: {anchor["radius"]}'
    )
    logger.debug(f"Additional Stores: {targets_count}")
    logger.debug(
        f'First Targets: ({len(target["coordinates"])}) {target["store"].upper()} Locations with Radius: {target["radius"]} '
    )

    local_points = []
    intermediate_points = []

    def compare_next_target(
        stores, current_point, current_radius, local_cluster, intermediate_cluster
    ):
        _len = len(stores)
        if _len > 1:

            next_radius = stores[0]["radius"]

            for next_point in stores[0]["coordinates"]:

                point_distance = gcc.distance_between_points(
                    current_point, next_point, unit="miles"
                )
                if point_distance < current_radius + next_radius:
                    local_cluster = [(next_point[1], next_point[0])]

                    intermediate_point = gcc.intermediate_point(
                        current_point,
                        next_point,
                        nom._intermediate_ratio(current_radius, next_radius),
                    )
                    intermediate_cluster += [
                        (intermediate_point[1], intermediate_point[0])
                    ]

                    return compare_next_target(
                        stores[1:],
                        next_point,
                        next_radius,
                        local_cluster,
                        intermediate_cluster,
                    )

        if _len == 1:
            next_radius = stores[0]["radius"]
            # print(stores[0]["store"])

            # print(current_radius, next_radius)
            for next_point in stores[0]["coordinates"]:
                point_distance = gcc.distance_between_points(
                    current_point, next_point, unit="miles"
                )
                if point_distance < current_radius + next_radius:
                    local_cluster += [(next_point[1], next_point[0])]

                    intermediate_point = gcc.intermediate_point(
                        current_point,
                        next_point,
                        nom._intermediate_ratio(current_radius, next_radius),
                    )
                    intermediate_cluster += [
                        (intermediate_point[1], intermediate_point[0])
                    ]

                    continue

        return local_cluster

    for anchor_point in anchor_points:
        local_cluster = [(anchor_point[1], anchor_point[0])]
        intermediate_cluster = []

        local_cluster += compare_next_target(
            targets, anchor_point, anchor_radius, local_cluster, intermediate_cluster
        )
        if len(local_cluster) > targets_count - 1:
            local_points += [local_cluster]

        if len(intermediate_cluster) > targets_count - 2:
            intermediate_points += [intermediate_cluster]

    return intermediate_points, local_points


t0 = time.perf_counter()
intermediate_points, local_points = int_cluster_points_multi(stores_dataframe_d)
t1 = time.perf_counter()

logger.debug(
    f"int_cluster_points_multi took {round((t1-t0) * 1000, 2)}ms and found {len(local_points)} clusters."
)

intermediate_points_dataframe = pd.DataFrame(intermediate_points)
intermediate_points_dataframe.head(5)

# local_points_dataframe = pd.DataFrame(local_points)
# local_points_dataframe

2022-10-25 14:24:30,579: 15: DEBUG: Using Store: (51) IKEA Locations as Anchor Points with Radius: 50
2022-10-25 14:24:30,580: 18: DEBUG: Additional Stores: 3
2022-10-25 14:24:30,580: 19: DEBUG: First Targets: (25) MICROCENTER Locations with Radius: 30 
2022-10-25 14:24:30,590: 104: DEBUG: int_cluster_points_multi took 10.76ms and found 21 clusters.


,0,1,2,3,4,5,6,7,8,9
0,"(42.274680628854114, -71.09707574838758)","(42.27149236650211, -71.47740159901544)","(42.27681474527256, -71.39793200016433)","(42.26221391996673, -71.53572614494159)","(41.92506351443206, -71.57761835926961)",None,None,None,None,None
1,"(40.94859926801059, -73.33746692723902)","(40.85862573384604, -73.84882761424436)","(40.82474415549997, -73.7778298497993)","(41.004244628035934, -73.87436731379944)","(40.960087088609086, -73.93552235148874)","(40.82174262532122, -74.02272488248843)","(40.95026079171274, -74.07017312154117)","(40.72160655910772, -74.13011977372967)","(40.85719910281167, -74.23877219689173)","(40.71996881130753, -74.37703266451399)"
2,"(40.75279277070353, -73.56529139835874)","(40.85862573384604, -73.84882761424436)","(40.82474415549997, -73.7778298497993)","(41.004244628035934, -73.87436731379944)","(40.960087088609086, -73.93552235148874)","(40.82174262532122, -74.02272488248843)","(40.95026079171274, -74.07017312154117)","(40.72160655910772, -74.13011977372967)","(40.85719910281167, -74.23877219689173)","(40.71996881130753, -74.37703266451399)"
3,"(40.736586430851865, -73.68933842313325)","(40.85862573384604, -73.84882761424436)","(40.82474415549997, -73.7778298497993)","(41.004244628035934, -73.87436731379944)","(40.960087088609086, -73.93552235148874)","(40.82174262532122, -74.02272488248843)","(40.95026079171274, -74.07017312154117)","(40.72160655910772, -74.13011977372967)","(40.85719910281167, -74.23877219689173)","(40.71996881130753, -74.37703266451399)"
4,"(40.71449079940363, -73.74556716261844)","(40.85862573384604, -73.84882761424436)","(40.82474415549997, -73.7778298497993)","(41.004244628035934, -73.87436731379944)","(40.960087088609086, -73.93552235148874)","(40.82174262532122, -74.02272488248843)","(40.95026079171274, -74.07017312154117)","(40.72160655910772, -74.13011977372967)","(40.85719910281167, -74.23877219689173)","(40.71996881130753, -74.37703266451399)"


##### Local Points Dataframe

In [33]:
local_points_dataframe = pd.DataFrame(local_points)
# local_points_dataframe.head(5)

## Cleaning Results

### Finding Centroid of Each Intermediate Point Cluster

#### Centroid Points Cluster Function
```centroid_points_cluster ```This function takes lists of (lat, lon) coordinates and returns a flat list of the centroid location of each cluster.

##### Intermediate Points Centroid Dataframe


In [34]:
def centroid_points_cluster(cluster: List[List[(Tuple[float, float])]]):
    cent_points: List[Tuple(float, float)] = []

    for region in cluster:
        if region:
            if len(region) == 1:
                cent_points += [region[0]]
            else:
                cent_point = nom._cent_points(region)
                cent_points += [cent_point]

    return cent_points

intermediate_points_centroids = centroid_points_cluster(intermediate_points)

intermediate_points_centroids_dataframe = pd.DataFrame(
    intermediate_points_centroids, columns=["Latitude", "Longitude"]
)
intermediate_points_centroids_dataframe.to_clipboard()
logger.debug(f"({len(intermediate_points_centroids_dataframe.index)}) Intermediate Points Clusters Found")
# intermediate_points_centroids_dataframe

2022-10-25 14:24:30,658: 20: DEBUG: (21) Intermediate Points Clusters Found


##### Local Points Centroid Dataframe


In [35]:
local_points_centroids = centroid_points_cluster(local_points)

local_points_centroids_dataframe = pd.DataFrame(
    local_points_centroids, columns=["Latitude", "Longitude"]
)
# local_points_centroids_dataframe.to_clipboard()

logger.debug(f"({len(local_points_centroids_dataframe.index)}) Local Points Clusters Found")
# local_points_centroids_dataframe.head()

2022-10-25 14:24:30,676:  8: DEBUG: (21) Local Points Clusters Found


## Mapping Results

#### Local Points Map

In [36]:
local_points_map = folium.Map(
    location=[
        local_points_centroids_dataframe.Latitude.mean(),
        local_points_centroids_dataframe.Longitude.mean(),
    ],
    zoom_start=4,
)

for point in local_points_centroids:
    folium.Circle([point[0], point[1]], nom._miles_to_meters(25)).add_to(
        local_points_map
    )

local_points_map

### Smoothing Cluster Array Function

#### Smoothing Intermediate Points Centroids
```smooth_cluster_array ```This function takes a list of (lat, lon) coordinates and groups adjacent locations together based on a radius in miles.

In [37]:
def smooth_cluster_array(coords: List, radius=10):
    clean_cluster = []
    c = coords.copy()
    _len = len(c)

    for i, p in enumerate(c):
        cluster_ = []

        if not any(p in x for x in clean_cluster):
            cluster_ += [(p[0], p[1])]

            for target in c[i + 1 :]:
                _dist = gcc.distance_between_points(
                    (p[1], p[0]), (target[1], target[0]), unit="miles"
                )

                if _dist < radius:
                    cluster_ += [(target[0], target[1])]

            clean_cluster += [cluster_]

    return clean_cluster

set_radius = 20

intermediate_smooth_cluster_ = smooth_cluster_array(
    intermediate_points_centroids, set_radius
)
logger.debug(
    f"{len(intermediate_points_centroids_dataframe.index)} Intermediate Points Centroids reduced to {len(intermediate_smooth_cluster_)} groups of locations within {set_radius} miles of each other."
)

smooth_dataframe = pd.DataFrame(intermediate_smooth_cluster_)
smooth_dataframe.head(5)

2022-10-25 14:24:30,733: 29: DEBUG: 21 Intermediate Points Centroids reduced to 11 groups of locations within 20 miles of each other.


,0,1,2,3,4,5,6,7
0,"(42.202180242788145, -71.41733370124949)",None,None,None,None,None,None,None
1,"(40.867026389238326, -73.96149369756337)","(40.84734338555655, -73.98412343428191)","(40.84568381539285, -73.9965381327324)","(40.84346054605072, -74.00216167701274)","(40.85292324540224, -74.00447323130788)","(40.84357406069147, -74.00810912509255)","(40.80652411794281, -74.0902425405823)","(40.81323188696448, -74.09639254665268)"
2,"(40.1266939957204, -75.52388122700027)",None,None,None,None,None,None,None
3,"(39.05532112349886, -77.0106489850327)","(39.037726332327985, -77.02764365635952)",None,None,None,None,None,None
4,"(40.069655393633234, -82.96567919527125)",None,None,None,None,None,None,None


#### Finding Centroids of Groups of Adjacent Intermediate Points

```IN ```Intermediate point centroids grouped together by proximity

```OUT ```Single list of points consisting of the average (centroid) location of each group

In [38]:
intermediate_smooth_cluster = centroid_points_cluster(intermediate_smooth_cluster_)

intermediate_smooth_cluster_dataframe = pd.DataFrame(
    intermediate_smooth_cluster, columns=["Latitude", "Longitude"]
)
# smooth_cluster_dataframe.to_clipboard()
logger.debug(f"({len(intermediate_smooth_cluster_dataframe.index)}) Points Found")
intermediate_smooth_cluster_dataframe.head(5)

2022-10-25 14:24:30,763:  7: DEBUG: (11) Points Found


,Latitude,Longitude
0,42.202180,-71.417334
1,40.839980,-74.017954
2,40.126694,-75.523881
3,39.046524,-77.019147
4,40.069655,-82.965679


### Mapping the Intermediate Point Centroids

```NOTE ```The overlapping circles are what get reduced in [Finding Centroids of Groups of Adjacent Intermediate Points](####Finding-Centroids-of-Groups-of-Adjacent-Intermediate-Points)

In [39]:
cent_points_map = folium.Map(
    location=[
        intermediate_points_centroids_dataframe.Latitude.mean(),
        intermediate_points_centroids_dataframe.Longitude.mean(),
    ],
    zoom_start=4,
)

for point in intermediate_points_centroids:
    folium.Circle([point[0], point[1]], nom._miles_to_meters(25)).add_to(
        cent_points_map
    )

cent_points_map

### Mapping Final Results with Intermediate Points Shown as a Polygon

In [40]:
smoothed_cluster_map = folium.Map(
    location=[
        intermediate_smooth_cluster_dataframe.Latitude.mean(),
        intermediate_smooth_cluster_dataframe.Longitude.mean(),
    ],
    zoom_start=4,
)

circle_format = {"color": "#f77f00", "fill": True, "fillOpacity": 0.5}
for point in intermediate_smooth_cluster:
    folium.Circle(
        [point[0], point[1]], nom._miles_to_meters(25), **circle_format
    ).add_to(smoothed_cluster_map)

file_out = "smoothed_map"

for store, setting in stores_settings.items():
  if setting["enabled"]:
    file_out += f"_{store[:4]}-{setting['radius']}"

for i, j in enumerate(intermediate_points):
    folium.Polygon(j).add_to(smoothed_cluster_map)

smoothed_cluster_map.save(f"../output/maps/{file_out}.html")

logger.info(f"Map file saved in output/maps/{file_out}.html")
smoothed_cluster_map

2022-10-25 14:24:30,834: 26: INFO: Map file saved in output/maps/smoothed_map_ikea-50_micr-30_cont-15_llbe-25.html
